# Speech recognition task
In this notebook I will guide you through one of the most common challenges of these days, that is, a speech recognition task. This is a **multiclassification** task, our dataset is composed of 105835.wav recordings with 35 possible labels and for each recording we have Mel-frequencies. First, we will see how to load data and how to process them. Then, we will use differnt algorithms to predict our classes. Particularly, we will deal with:

1. **Decision forest**
2. **CNN 2D**
3. **CNN 1D**

### Data processing

The first thing to do is to import all the necessary libraries:

In [1]:
import pandas as pd
import numpy as np
from numpy import save

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


Now we can start to load the data and then to precess them. Before starting, let's see in more details our data, which you can find here: https://surfdrive.surf.nl/files/index.php/s/A91xgk7B5kXNvfJ

- **feat.npy** is an array with Mel-frequency coefficients extracted from each wav file. The features at index *i* in this array were extracted from the wav file at index *i* of the array in the file path.npy.
- **path.npy** is an array with the order of wav files in the feat.npy array.
- **train.csv** contains two columns: path with the filename of the recording and word with word which was pronounced in the recording. This is the training portion of the data.
- **test.csv** is the testing portion of the data, and it has the same format as the file train.csv except that the column word is absent.

For transparency reason I also described the test portion of the data. However, I do not have access to its label so for now on I will not use it. We will divide our data in training and validation sets, which is not a problem because we have enough data.

In [2]:
                        # --------------------- LOAD DATA ------------------------ #

features = np.load("feat.npy", allow_pickle = True)
path = np.load("path.npy", allow_pickle = True)
train = pd.read_csv("train.csv", delimiter = ",")
#test = pd.read_csv("test.csv", delimiter = ",")

Tu put it simply, in path.npy we have the "wav name" of the recordings and in feat.npy we have its features.

In [3]:
print("The file with name ",path[0], "has this set of features:")
print("----------------------------------------------------------")
print(features[0])

The file with name  8d37d10e7f97ddea2eca9d39a4cf821b4457b041.wav has this set of features:
----------------------------------------------------------
[[-1.0160675e+01 -1.3804866e+01  9.1880971e-01 ...  1.5415058e+00
   1.1875046e-02 -5.8664594e+00]
 [-9.9697800e+00 -1.3823588e+01 -7.0778362e-02 ...  1.5948311e+00
   4.3481258e-01 -5.1646194e+00]
 [-9.9518738e+00 -1.2771760e+01 -1.2623003e-01 ...  3.4290311e+00
   2.7361808e+00 -6.0621500e+00]
 ...
 [-9.9772453e+00 -1.8227636e+01  5.4514462e-01 ... -2.4311361e+01
  -1.2033378e+01 -3.0492477e+00]
 [-9.8815651e+00 -1.3674727e+01  5.1737971e+00 ... -1.0951551e+01
  -2.6184993e+00  1.5815194e+00]
 [-1.0164914e+01 -1.2636200e+01  4.4198503e+00 ... -5.7269883e+00
   4.0758374e-01 -4.4462643e+00]]


As you can see the index in path and features must be the same, otherwise we risk misaligning "wav file name - features" and consequently, have a very bad accuracy.

Ok, now stay with me, because this is the most boring part of the project. Unfortunatly the structure of the data is not suitable to apply ML or DL algorithms, but this is fine, because we are Data Scientists and we know that we have to get "our hands dirty" before having fun ;).

The "problem" is that we were given the indices of train set and test set (see train.csv and test.csv). I know, now you are thinking: "Why in the world is this a problem? We need those!". That is true, but the problem is that the indices of train set and test set only list their "file name" but not their features.

In [4]:
#print(test.head())
print()
print(train.head())


                                           path word
0  8d37d10e7f97ddea2eca9d39a4cf821b4457b041.wav  one
1  9a8f761be3fa0d0a963f5612ba73e68cc0ad11ba.wav  one
2  314cdc39f628bc68d216498b2080bcc7a549a45f.wav  one
3  cc499e63eee4a3bcca48b5b452df04990df83570.wav  one
4  38cdcc4d9432ce4a2fe63e0998dbca91e64b954a.wav  one


In conclusion, we need to find a way to split our data, which are divided in path.npy and feat.npy, according to the indices in train.csv and test.csv.

To do that, we first link our path.npy and feat.npy with a dictionary, key = path and value = feat. You will understand why in a moment. 

In [5]:
# create dictionary: key = path and value = feat
dic = {} 
for i in range(len(path)):
    dic[path[i]] = features[i]

After creating a dictionary, we need a fuction to split all our data (now in the dictionary) into train and test sets according to the csv files. Basically, we go through all the data_frames, which are train.csv and test.csv and thanks to the information from the dictionary we can create a list for each data_frame, where in the *ith* position for both, data_frame and dictionary, we have its *i* set of features. 

In [6]:
# this function take as argument a pandas data frame and a dictionary
# and create a new list according to the ith position in the dataframe

def create_list(data_frame,dic):
    new_list= []
    for i in range(len(data_frame)):
        if data_frame["path"][i] in dic.keys():
            new_list.append(dic[data_frame["path"][i]]) # in the position i we add its features thanks to the dic
    return new_list

#in order to convert a list in a numpy array we need to padd our data
def padding(data):
    zeros_list=[0,0,0,0,0,0,0,0,0,0,0,0,0]
    for example in range(len(data)):
        if data[example].shape[0]!=99:
            to_change=data[example].tolist()
            for adding in range(99-len(to_change)):
                to_change.append(zeros_list)
            data[example]=np.array(to_change)     
    return data
    

Now we can apply all the defined functions and we can also get an overview of the shape so that we can sense the kinds of data we are going to deal with.

In [7]:
# split test and train 
training_data = create_list(train,dic)
#test_data = create_list(test,dic)

# padding
training_data = padding(training_data)
#test_data = padding(test_data)

# convert to array
training_data = np.array(training_data)
#test_data = np.array(test_data)

#check shape
training_data.shape #,test_data.shape

(94824, 99, 13)

We did it! We have now a tidy training data composed of 94824 examples with 99 lists of 13 elements each. We can now link the features in training_data to the file name in train.csv and the features in test_data to the file name in test.csv.

In [47]:
print("This file name\n", train.loc[0], "\n\n has these features \n" )# with .loc I can see all the columns of
                                                                    # the index that I passed. See pandas documentation
                                                                    # for more details.
print(training_data[0])

This file name
 path    8d37d10e7f97ddea2eca9d39a4cf821b4457b041.wav
word                                             one
Name: 0, dtype: object 

 has these features 

[[-1.01606750e+01 -1.38048658e+01  9.18809712e-01 ...  1.54150581e+00
   1.18750464e-02 -5.86645937e+00]
 [-9.96977997e+00 -1.38235884e+01 -7.07783625e-02 ...  1.59483111e+00
   4.34812576e-01 -5.16461945e+00]
 [-9.95187378e+00 -1.27717600e+01 -1.26230031e-01 ...  3.42903113e+00
   2.73618078e+00 -6.06215000e+00]
 ...
 [-9.97724533e+00 -1.82276363e+01  5.45144618e-01 ... -2.43113613e+01
  -1.20333776e+01 -3.04924774e+00]
 [-9.88156509e+00 -1.36747274e+01  5.17379713e+00 ... -1.09515514e+01
  -2.61849928e+00  1.58151937e+00]
 [-1.01649141e+01 -1.26362000e+01  4.41985035e+00 ... -5.72698832e+00
   4.07583743e-01 -4.44626427e+00]]


We are almost at the end of this chapter, but first we need to encode our target. We will use the function [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html), because our target is a string and it is much easier to work with integer and so we encode our labels into numbers.

In [9]:
# encode the target
train_numpy = train.values # from pandas to numpy
labels = train_numpy[:,1] # get the labels
encoder = LabelEncoder() # generate the encoder
encoder.fit(labels) # fit the encoder
labels = encoder.transform(labels) # transform the labels into integers

In order to avoid doing this data processing again in the future, let's save these new data.

In [10]:
## SAVE PROCESSED DATA
save('training_data.npy', training_data)
save("labels.npy",labels)

### Decision forest

We create now a validation data for testing the decision forest thanks to the function [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [11]:
## SPLITTING
X_train, X_test, y_train, y_test = train_test_split(training_data, labels, test_size=0.2, random_state=42)

In [12]:
# check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(75859, 99, 13)
(75859,)
(18965, 99, 13)
(18965,)


#### Features engineering

In order to improve the accuracy of our decision forest, it is wise to do some feature engineering first. We define two functions. The first creates the mean of our signal, while the second does something a little bit more complicated, that is, it applys different statistics indices to our signal.

In [13]:
def features_mean(signal):
    return np.mean(signal,axis=2)

def features(signal, functions):
    summaries=[]
    for fn in functions:
        summaries.append(fn(signal,axis=2))
    return np.concatenate(summaries,axis=1)

summaries = [np.mean, np.min, np.max, np.std]

X_train_summaries = features(X_train, summaries)
X_test_summaries = features(X_test, summaries)

X_train_mean = features_mean(X_train)
X_test_mean = features_mean(X_test)

#### Running the model

Ready to have some fun? Here I create a function in order to run the decision forest with different numbers of nodes. As you know, the number of nodes in a forest is extremely important as it allows to control for overfitting. Let's take a look at all the arguments:

- **n_estimators** : this is a list and as I mentioned before, with this we can control the number of nodes
- **X_train and X_test** : as we used 2 different kinds of feature engineering, here we can choose which one to use
- **y_train and y_test** : regardless of wich kind of training data we use, the test set remains the same and therefore these arguments have already default values
- **random_state** : a simple way to control randomness when looking for the best split at each node. This is useful to reproduce the accuracy.

I leave you [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) the official documentation about decision forest.

In conclusion, not only does this function allow us to choose the training data and the number of nodes, but it also saves the accuracy for each number of nodes.

In [14]:
## this function take as argument a training and a validation data set and return the accuracy based on 
# the number of nodes, which is encode here as n_estimators

def run_forest(n_estimators ,X_train, X_test, y_train = y_train, y_test = y_test , random_state = 333):
    
    acc=[] # list of accuracy which depends on the hyperparameter n_estimators

    for num_features in n_estimators: # hyperparameters to change with higher number
        forest = RandomForestClassifier(n_estimators=num_features, 
                               bootstrap = True,
                               max_features = 'sqrt',
                                random_state = random_state)
        fitted_model=forest.fit(X_train,y_train)
        prediction=fitted_model.predict(X_test)
        accuracy=accuracy_score(y_test,prediction)
        solution = (num_features,accuracy)
        acc.append(solution)
        
    return acc

Let's run the model, here we can define the lists to put as **n_estimators** argument. My machine cannot handle all these data, for the sake of the example, I just put 1 node for each type of features engineering. You can try with [50,100,200]  and you will see that the accuracy is going to increase, although it cannot go over 20%. Not too much eh? For this we have DL ;).

In [15]:
#set up different number of nodes 
n_estimators_mean = [1]
n_estimators_summaries = [1]

In [16]:
# accuracy for the mean
accuracy_mean = run_forest(n_estimators_mean,X_train_mean,X_test_mean)
print(accuracy_mean)

[(1, 0.05715792248879515)]


In [17]:
# accuracy for the summaries
accuracy_summaries = run_forest(n_estimators_summaries,X_train_summaries,X_test_summaries)
print(accuracy_summaries)

[(1, 0.07835486422356973)]


### 2D Convolutional Neural Network 

Ok, now the moment that all of us were waiting, let's apply Convolutional Neural Network. In this case, we will apply the 2D layer, which is mainly used for image classification, but also in this case we can reach an important accuracy (more than 90%) by reshaping the dimensions of our input shape. For creating this model, I was inspired by this very well done [tutorial](https://www.datacamp.com/community/tutorials/convolutional-neural-networks-python) on DataCamp.

First of all, let's upload the data in new variables so that we have everything nice and clean.

In [18]:
# change name of variables 
train_X = training_data
#test_X = test_data
train_Y = labels

As I said before, we need to reshape the input space, [reshape function](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html) can help us.

In [20]:
# reshape in four dimensions for input CNN
train_X = train_X.reshape(-1, 99,13, 1)
#test_X = test_X.reshape(-1, 99,13, 1)
train_X.shape #, test_X.shape

(94824, 99, 13, 1)

In [21]:
# transform data type in float
train_X = train_X.astype('float32')
#test_X = test_X.astype('float32')

With CNN we are going to predict a probability for each possible class for each example and for this we need to change again our target. We will use the [to_categorical](https://keras.io/utils/) function. This function creates for each target a list with as many position as number of classes-1 (because it starts from 0) and each *ith* position represents the *ith* class.

In [22]:
# transform the labels
train_Y_one_hot = to_categorical(train_Y)

For example, here we can see that the first target is the 22° class, this is because in the 21th position we have 1, while in the others we have 0.

In [23]:
train_Y_one_hot[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)

Before running the model we need of course to create a validation set.

In [24]:
## CREATE THE VALIDATION SET 
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, 
                                                           random_state=13)

One last check.

In [25]:
# check all the shape
train_X.shape,valid_X.shape,train_label.shape,valid_label.shape

((75859, 99, 13, 1), (18965, 99, 13, 1), (75859, 35), (18965, 35))

#### First attempt

As you might know, before running CNN we need to set up some hyperparameters.

In [26]:
# set up hyperparameters 
batch_size = 64
epochs = 1
num_classes = 35 # fix
np.random.seed(222)

Here our model: 3 CNN2D layers and a fully connected layer before the output layer.

In [29]:
# set up the layers 
fashion_model = Sequential()

fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(99,13,1),padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))

fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))  
fashion_model.add(Dense(num_classes, activation='softmax'))

In [30]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, 
                      optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [31]:
## check the summary
fashion_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 99, 13, 32)        320       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 99, 13, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 50, 7, 32)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 50, 7, 64)         18496     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 50, 7, 64)         0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 25, 4, 64)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 25, 4, 128)        73856     
__________

Let's run. Also here, I just use one epoch for the sake of the example. You should try with at least 20 epochs.

In [33]:
## train and test the accuracy in the validation set
fashion_train = fashion_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,
                                  verbose=1,validation_data=(valid_X, valid_label))

Train on 75859 samples, validate on 18965 samples
Epoch 1/1
75859/75859 [==============================] - 263s 3ms/step - loss: 1.0299 - acc: 0.7024 - val_loss: 0.6447 - val_acc: 0.8129


In conclusion, with this model you should reach 85% accuracy in validation. Not bad eh? However, we can go much more further.

#### Second attempt

Here we try to control overfitting by introducing the Dropout function and we hope to improve accuracy. Spoiler alert: we do!

To put it simply, during training, some number of layer outputs are randomly ignored or "dropped out." For this we use less parameters and as a result we can control overfitting.

In [34]:
# set up hyperparameters 
batch_size = 124
epochs = 1
num_classes = 35 # fix
np.random.seed(222)

In [36]:
## set up the dropout to improve accuracy
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(99,13,1),padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.4))

fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))  
fashion_model.add(Dropout(0.3))
fashion_model.add(Dense(num_classes, activation='softmax'))

In [37]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, 
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

In [38]:
fashion_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 99, 13, 32)        320       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 99, 13, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 50, 7, 32)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 50, 7, 32)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 50, 7, 64)         18496     
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 50, 7, 64)         0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 25, 4, 64)         0         
__________

In [39]:
fashion_train = fashion_model.fit(train_X, train_label, 
                                  batch_size=batch_size,epochs=epochs,verbose=1,
                                  validation_data=(valid_X, valid_label))

Train on 75859 samples, validate on 18965 samples
Epoch 1/1
75859/75859 [==============================] - 343s 5ms/step - loss: 2.4086 - acc: 0.3346 - val_loss: 1.2260 - val_acc: 0.6632


In conclusion, with this model you should reach at least 90% accuracy by increasing the number of epochs, let's say a number between 20 and 50.

### 1D Convolutional Neural Network

Althoug, we can reach a good accuracy with 2D layers, the 1D layer turned out to have better accuracy for this data. This is not surprusing, 1D CNN it is widely used for speech recognition. So, let's get started.

Upload data, reshape and transform the targets.

In [40]:
#go back to two three dimensions
train_X = training_data
#test_X = test_data
train_Y = labels

train_X = train_X.reshape(-1, 99,13)
#test_X = test_X.reshape(-1, 99,13)

# transform the labels
train_Y_one_hot = to_categorical(train_Y)

train_X.shape #, test_X.shape

(94824, 99, 13)

Create validation set.

In [41]:
## CREATE THE VALIDATION SET 
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, 
                                                           random_state=13)

Hyperparameters tuning.

In [42]:
# set up hyperparameters 
batch_size = 256
epochs = 1
num_classes = 35 # fix
np.random.seed(222)

As you can see, this is a very deep model. My group and I spent a lot of time tuning this model because we knew we could reach a very high accuracy. Eventually we were able to reach 95% but I know that it is also possible to get 98%. I leave this challenge to you.

In [43]:
fashion_model = Sequential()

fashion_model.add(Conv1D(64, kernel_size=6,activation='relu',padding='same',input_shape=(99,13)))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))

fashion_model.add(Dropout(0.2))
fashion_model.add(Conv1D(128, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))

fashion_model.add(Conv1D(128, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(128, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))


fashion_model.add(Conv1D(128, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(256, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(256, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(256, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(512, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(1024, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(1024, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Flatten())
fashion_model.add(Dense(1024, activation='relu'))
fashion_model.add(LeakyReLU(alpha=0.1))           
fashion_model.add(Dropout(0.2))
fashion_model.add(Dense(num_classes, activation='softmax'))

In [44]:
fashion_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 99, 64)            5056      
_________________________________________________________________
batch_normalization_1 (Batch (None, 99, 64)            256       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 99, 64)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 64)            0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 50, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 128)           49280     
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 128)           512       
__________

In [45]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, 
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

In [46]:
fashion_train = fashion_model.fit(train_X, train_label, 
                                  batch_size=batch_size,epochs=epochs,verbose=1,
                                  validation_data=(valid_X, valid_label))

Train on 75859 samples, validate on 18965 samples
Epoch 1/1
75859/75859 [==============================] - 1574s 21ms/step - loss: 2.5870 - acc: 0.2630 - val_loss: 1.7327 - val_acc: 0.5078


If I remember correctly, we trained our model for 100 epochs. As always here, for the sake of the example I just ran it for 1 epoch.

## Conclusion

In this notebook we have discussed three different ways of dealing with a speech classification task. First, we tried to solve the problem with a ML algorithm, however, we could not go over 20% accuracy, which is normal becuase this kinds of tasks require to apply DL methods. Therefore, two different types of CNN layers were applied: 2D and 1D. As a result a very high accuracy was reached, although even an higher one is possible to achive.

What do we bring home?
- These kinds of tasks cannot be solve with classic ML algortihms
- A lot of time is recquired for tuning DL models and unfortunately we do not know which one is the best. Try and error is often the standard even if academic and industrial studies can give us some guidelines